In [7]:
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl==0.15.2 peft accelerate bitsandbytes
!pip install datasets pandas

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-i7uxunh2/unsloth_79c7da0cac3c4d349a60f14c44ccf045
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-i7uxunh2/unsloth_79c7da0cac3c4d349a60f14c44ccf045
  Resolved https://github.com/unslothai/unsloth.git to commit a78b86e5c9c08b90f53a4ef89e6b9c6860fe66dc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.7/166.7 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.4 MB/s eta 0:00

## Defining the Model to be Finetuned

In [11]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct-unsloth-bnb-4bit" ,
    max_seq_length = 8192,
    load_in_4bit = False
)

==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.54.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

## Loading the dataset

In [12]:
from datasets import load_dataset

Bhojpuri_dataset = load_dataset("SatyamDev/alpaca_data_cleaned_bhojpuri", split="train")

README.md: 0.00B [00:00, ?B/s]

alpaca_data_cleaned_bhojpuri.csv:   0%|          | 0.00/95.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [13]:
Bhojpuri_dataset[0]

{'instruction': 'स्वस्थ रहे खातिर तीन गो टिप्स दीं।',
 'input': None,
 'output': '1. संतुलित अवुरी पौष्टिक आहार खाईं: सुनिश्चित करीं कि आपके भोजन में कई प्रकार के फल अवुरी सब्जी, दुबला प्रोटीन, साबुत अनाज अवुरी स्वस्थ वसा शामिल होखे। एहसे आपके शरीर के सबसे निमन तरीका से काम करे खाती जरूरी पोषक तत्व मिले में मदद मिलेला अवुरी पुरान बेमारी से बचाव में मदद मिल सकता।\n\n2. नियमित शारीरिक गतिविधि में शामिल होखे के चाही: हड्डी, मांसपेशी अवुरी हृदय संबंधी स्वास्थ्य के मजबूत बनावे खाती व्यायाम बहुत जरूरी बा। हर हफ्ता कम से कम 150 मिनट मध्यम एरोबिक व्यायाम भा 75 मिनट के जोरदार व्यायाम के लक्ष्य राखीं.\n\n3. पर्याप्त नींद लेवे के बा : शारीरिक अवुरी मानसिक भलाई खाती पर्याप्त गुणवत्ता वाला नींद बहुत जरूरी बा। इ मूड के नियंत्रित करे में मदद करेला, संज्ञानात्मक कामकाज में सुधार करेला अवुरी स्वस्थ विकास अवुरी प्रतिरक्षा के कामकाज के समर्थन करेला। हर रात 7-9 घंटा नींद के लक्ष्य राखीं।'}

### Changing the format of data into conversation format that model understands and requires.

In [14]:
def generate_conversation(convo):
  inputs = convo['instruction']
  outputs = convo['output']

  conversation = []
  for input, output in zip(inputs, outputs):
    conversation.append([
        {"role" : "user", "content" : input},
        {"role" : "assistant", "content" : output}
    ])
  return {"conversation" : conversation}

In [15]:
result = generate_conversation(Bhojpuri_dataset[0])
result

{'conversation': [[{'role': 'user', 'content': 'स'},
   {'role': 'assistant', 'content': '1'}],
  [{'role': 'user', 'content': '्'}, {'role': 'assistant', 'content': '.'}],
  [{'role': 'user', 'content': 'व'}, {'role': 'assistant', 'content': ' '}],
  [{'role': 'user', 'content': 'स'}, {'role': 'assistant', 'content': 'स'}],
  [{'role': 'user', 'content': '्'}, {'role': 'assistant', 'content': 'ं'}],
  [{'role': 'user', 'content': 'थ'}, {'role': 'assistant', 'content': 'त'}],
  [{'role': 'user', 'content': ' '}, {'role': 'assistant', 'content': 'ु'}],
  [{'role': 'user', 'content': 'र'}, {'role': 'assistant', 'content': 'ल'}],
  [{'role': 'user', 'content': 'ह'}, {'role': 'assistant', 'content': 'ि'}],
  [{'role': 'user', 'content': 'े'}, {'role': 'assistant', 'content': 'त'}],
  [{'role': 'user', 'content': ' '}, {'role': 'assistant', 'content': ' '}],
  [{'role': 'user', 'content': 'ख'}, {'role': 'assistant', 'content': 'अ'}],
  [{'role': 'user', 'content': 'ा'}, {'role': 'assistant'

### Getting the template used by llama3

In [16]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama3"
)

### Applying the template to our tokenizer

In [17]:
final_conversation = tokenizer.apply_chat_template(
    Bhojpuri_dataset.map(generate_conversation, batched = True)["conversation"],
    tokenize = False
)

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

## Converting conversation into Huggingface dataset

In [18]:
# It is important to convert into HuggingFace Dataset because the SFT Trainer
# requires train_dataset to be a HuggingFace Dataset

# Selectinge only the first 10000 conversations
conversation_subset = final_conversation[:10000]

# Adding column "text" because SFT Trainer requires this format
import pandas as pd
data = pd.Series(conversation_subset, name = "text")

from datasets import Dataset
final_dataset = Dataset.from_pandas(pd.DataFrame(data))

final_dataset = final_dataset.shuffle(seed=2000)

In [19]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,       # Choose any number > 0. Suggested values = 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Generally said that it's best to choose alpha = rank or rank X 2
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None

)

Unsloth 2025.8.1 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [20]:
!pip install transformers

## Defining the Training Parameters

In [21]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    max_steps=200,
    learning_rate=2e-4,
    logging_steps=5,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed = 3407,
    report_to="none"
)

## Defining SFT Trainer

In [22]:
from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=final_dataset,  # should be a HuggingFace Dataset with a "text" column
    dataset_text_field="text",
    args=training_args,
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [23]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 11,272,192 of 1,247,086,592 (0.90% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,2.485900
10,2.004500
15,1.978700
20,1.933500
25,1.909500
30,1.943500
35,1.772300
40,1.818100
45,1.701600
50,1.570200


In [24]:
# Testing the finetuned model
FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": "स्वस्थ रहे खातिर तीन गो टिप्स दीं।"}
]

inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

outputs = model.generate(inputs=inputs, max_new_tokens=512, use_cache=True)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nस्वस्थ रहे खातिर तीन गो टिप्स दीं।<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n1. स्वस्थ आहार खाए के: स्वस्थ आहार खाना खातिर एगो महत्वपूर्ण कदम हवे जेह में पौष्टिक आ प्राकृतिक खाद्य पदार्थ शामिल होखे जइसे कि फल, सब्जी, साबुत अनाज आ दू गो प्रोटीन स्रोत। स्वस्थ आहार खाना खातिर एगो साफ आहार नीति के जरूरत होला जवना में नियमित भोजन, कम कैलोरी खाना आ स्वस्थ व्यायाम शामिल होखे।\n\n2. नियमित व्यायाम करीं: नियमित व्यायाम स्वस्थ रहे आ जीवंत होखे खातिर एगो जरूरी कदम हवे जेह में प्रतिदिन कुछ घंटा के व्यायाम शामिल होखे। व्यायाम के कारण कोर के मांसपेशियों के मजबूती में मदद करेला, स्वस्थ रक्त प्रवाह में सुधार करेला, आ स्वस्थ वजन कम करेला। व्यायाम के जरूरत के निर्धारित करे खातिर आपन लक्ष्य आ स्वास्थ्य के जरूरत के मुताबिक नियमित व्यायाम के योजना बनाईं।\n\n3. नियमित ब्रेक लीं: नियमित ब्रेक लीं के स्वस्थ आहार आ व्यायाम के बीच के प्रतिक्रिया के संतुलित रूप से संभालल जा सकेला। ब्रेक लिहला से शरीर के संतुलन के बनावे में मदद मिलेला,